In [8]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import imageio


In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
samples_dir = 'samples_png'
image_files = os.listdir(samples_dir)
images = []
labels = []
# Define a dictionary to map symbol names to integers
symbol_dict = {'dot': 0, 'circle': 1, 'ex': 2, 'plus': 3, 'blank': 4}

for image_file in image_files:
    symbol_name = image_file.split('_')[0]  # Extract the symbol name from the filename
    img = Image.open(os.path.join(samples_dir, image_file)).convert('L')
    img = np.array(img)
    img = img / 255.0  # Trim to 7x7 centered and normalize pixel values
    images.append(img)
    label = symbol_dict[symbol_name]  # Map the symbol name to an integer
    labels.append(label)

images = np.array(images).reshape((len(images), 15, 15, 1))  # Reshape the images to add a channel dimension
labels = to_categorical(labels, num_classes=5)  # One-hot encode the labels

# Create an image data generator object
datagen = ImageDataGenerator(
    fill_mode='constant',
    cval=1.0,  # Value used for fill_mode = "constant"
    width_shift_range=4,  # Shift the image left or right by 7 pixels
    height_shift_range=4  # Shift the image up or down by 7 pixels
)

# Fit the data generator to your data
datagen.fit(images)

# Use the data generator to create augmented images
augmented_images, augmented_labels = datagen.flow(images, labels, batch_size=len(images), shuffle=False).next()

# Save the augmented images to a directory
save_dir = 'augmented_images'
os.makedirs(save_dir, exist_ok=True)

for i, (img, label) in enumerate(zip(augmented_images, augmented_labels)):
    # Reshape the image back to 2D and convert to 8-bit unsigned integer format before saving
    img_2d = (img.reshape((15, 15)) * 255).astype(np.uint8)
    # Get the symbol name from the label
    symbol_name = list(symbol_dict.keys())[list(symbol_dict.values()).index(np.argmax(label))]
    # Save the image with the symbol name first
    imageio.imsave(os.path.join(save_dir, f'{symbol_name}_augmented_image_{i}.png'), img_2d)

In [11]:
samples_dir = ['samples_png', 'augmented_images']
images = []
labels = []
# Define a dictionary to map symbol names to integers
symbol_dict = {'dot': 0, 'circle': 1, 'ex': 2, 'plus': 3, 'blank': 4}

for dir in samples_dir:
    image_files = os.listdir(dir)
    for image_file in image_files:
        symbol_name = image_file.split('_')[0]  # Extract the symbol name from the filename
        img = Image.open(os.path.join(dir, image_file)).convert('L')
        img = np.array(img)
        img = img[4:11, 4:11] / 255.0  # Normalize pixel values
        images.append(img)
        label = symbol_dict[symbol_name]  # Map the symbol name to an integer
        labels.append(label)

images = np.array(images)
labels = to_categorical(labels, num_classes=5)  # One-hot encode the labels
print(images.shape) 
print(labels.shape)

(407, 7, 7)
(407, 5)


In [167]:
# Define the neural network architecture
model = Sequential([
    Flatten(input_shape=(7, 7)),
    Dense(37, activation='sigmoid'),
    Dense(5, activation='sigmoid')
])
# Print the model summary
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 49)                0         
                                                                 
 dense_16 (Dense)            (None, 37)                1850      
                                                                 
 dense_17 (Dense)            (None, 5)                 190       
                                                                 
Total params: 2040 (7.97 KB)
Trainable params: 2040 (7.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [168]:
# Split the images and labels into a training set and a test set
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [169]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [170]:
# Train the model
model.fit(train_images, train_labels, epochs=400)

Epoch 1/400
11/11 [==============================] - 0s 1ms/step - loss: 1.6119 - accuracy: 0.2400
Epoch 2/400
11/11 [==============================] - 0s 2ms/step - loss: 1.5560 - accuracy: 0.3292
Epoch 3/400
11/11 [==============================] - 0s 1ms/step - loss: 1.5173 - accuracy: 0.4215
Epoch 4/400
11/11 [==============================] - 0s 1ms/step - loss: 1.4833 - accuracy: 0.4308
Epoch 5/400
11/11 [==============================] - 0s 1ms/step - loss: 1.4531 - accuracy: 0.5138
Epoch 6/400
11/11 [==============================] - 0s 1ms/step - loss: 1.4226 - accuracy: 0.5446
Epoch 7/400
11/11 [==============================] - 0s 2ms/step - loss: 1.3958 - accuracy: 0.5508
Epoch 8/400
11/11 [==============================] - 0s 1ms/step - loss: 1.3700 - accuracy: 0.5508
Epoch 9/400
11/11 [==============================] - 0s 1ms/step - loss: 1.3460 - accuracy: 0.5262
Epoch 10/400
11/11 [==============================] - 0s 2ms/step - loss: 1.3221 - accuracy: 0.5508
Epoch 11/

In [171]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

3/3 [==============================] - 0s 3ms/step - loss: 0.3590 - accuracy: 0.8780
Test accuracy: 0.8780487775802612


In [200]:
# Get the weights of the model
weights = model.get_weights()[0] #solo pesos 1ra capa
bias = model.get_weights()[1] #solo bias 1ra capa
# print(len(weights[48]))
# print(len(bias))

# print(range(len(weights)))

with open('weights_and_bias.txt', 'w') as f:
    f.write("{")
    for j in range(len(weights[0])):
        for i in range(len(weights)):
            f.write(f"{np.round(weights[i][j]*255).astype(int)}, ")
        f.write(f"{np.round(bias[j]*2**13).astype(int)}, ")


# Get the weights of the model
weights = model.get_weights()[2] #solo pesos 1ra capa
bias = model.get_weights()[3] #solo bias 1ra capa
print(len(weights[36]))
print(len(bias))
with open('weights_and_bias.txt', 'a') as f:
    for j in range(len(weights[0])-1):
            for i in range(len(weights)):
                f.write(f"{np.round(weights[i][j]*255).astype(int)}, ")
            f.write(f"{np.round(bias[j]*2**13).astype(int)}, ")
    f.write("}")

print(f"{np.round(weights[0][0]*255)}, {np.round(weights[1][0]*255)}, {np.round(bias[0]*2**13)}")
print(f"{np.round(weights[0][1]*255)}, {np.round(weights[1][1]*255)}, {np.round(bias[1]*2**13)}")
print(f"{np.round(weights[0][3]*255)}, {np.round(weights[1][3]*255)}, {np.round(bias[3]*2**13)}")


5
5
266.0, 221.0, 672.0
176.0, -526.0, 81.0
-99.0, 309.0, -929.0


In [201]:
# Get the weights of the model
weights = model.get_weights()[0] #solo pesos 1ra capa
bias = model.get_weights()[1] #solo bias 1ra capa

with open('weights_and_bias_fp.txt', 'w') as f:
    f.write("{")
    for j in range(len(weights[0])):
        for i in range(len(weights)):
            f.write(f"{weights[i][j]}, ")
        f.write(f"{bias[j]}, ")

# Get the weights of the model
weights = model.get_weights()[2] #solo pesos 1ra capa
bias = model.get_weights()[3] #solo bias 1ra capa

with open('weights_and_bias_fp.txt', 'a') as f:
    for j in range(len(weights[0])-1):
        for i in range(len(weights)):
            f.write(f"{weights[i][j]}, ")
        f.write(f"{bias[j]}, ")
    f.write("}")

In [202]:
# Get the weights of the model
weights = model.get_weights()[0] #solo pesos 1ra capa
bias = model.get_weights()[1] #solo bias 1ra capa

with open('weights_and_bias_hex_fp.txt', 'w') as f:
    f.write("{")
    for j in range(len(weights[0])):
        for i in range(len(weights)):
            f.write(f"{format(np.frombuffer(np.float32(weights[i][j]).tobytes(), 'uint32')[0], '08x')}, ")
        f.write(f"{format(np.frombuffer(np.float32(bias[j]).tobytes(), 'uint32')[0], '08x')}, ")

# Get the weights of the model
weights = model.get_weights()[2] #solo pesos 1ra capa
bias = model.get_weights()[3] #solo bias 1ra capa

with open('weights_and_bias_hex_fp.txt', 'a') as f:
    for j in range(len(weights[0])-1):
        for i in range(len(weights)):
            f.write(f"{format(np.frombuffer(np.float32(weights[i][j]).tobytes(), 'uint32')[0], '08x')}, ")
        f.write(f"{format(np.frombuffer(np.float32(bias[j]).tobytes(), 'uint32')[0], '08x')}, ")
    f.write("}")

In [194]:
weights = model.get_weights()[2] #solo pesos 1ra capa
bias = model.get_weights()[3] #solo bias 1ra capa
print(np.round(weights[:,3]*255).astype(int))
print(np.round(bias[3]*2**13).astype(int))

[ -99  309   91   90  132  178 -161 -153  -37 -293  161 -191 -246 -126
  228   65  130   15  -93  227   22  -89  -58  -79  282   29 -226 -251
   83   39 -151 -166   59   90  -50   -4 -266]
-929


In [203]:
with open('weights_and_bias.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers, reverse the list, and join it back into a string
reversed_data = ['{'+', '.join(line.strip('{}').split(', ')[::-1])+'}' for line in data]

# Join the reversed lines into a single string with newline characters between them
reversed_data = '\n'.join(reversed_data)

# Write the reversed string to the file
with open('weights_and_bias.txt', 'w') as f:
    f.write(reversed_data)

#######################################################
with open('weights_and_bias_hex_fp.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers, reverse the list, and join it back into a string
reversed_data = ['{'+', '.join(line.strip('{}').split(', ')[::-1])+'}' for line in data]

# Join the reversed lines into a single string with newline characters between them
reversed_data = '\n'.join(reversed_data)

# Write the reversed string to the file
with open('weights_and_bias_hex_fp.txt', 'w') as f:
    f.write(reversed_data)


#######################################################
with open('weights_and_bias_fp.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers, reverse the list, and join it back into a string
reversed_data = ['{'+', '.join(line.strip('{}').split(', ')[::-1])+'}' for line in data]

# Join the reversed lines into a single string with newline characters between them
reversed_data = '\n'.join(reversed_data)

# Write the reversed string to the file
with open('weights_and_bias_fp.txt', 'w') as f:
    f.write(reversed_data)

In [204]:
with open('weights_and_bias.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers and count the numbers
count = [len(line.strip('{}').split(', ')) for line in data]

print(count)


with open('weights_and_bias_hex_fp.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers and count the numbers
count = [len(line.strip('{}').split(', ')) for line in data]

print(count)



with open('weights_and_bias_fp.txt', 'r') as f:
    data = f.read().split('\n')

# Split each line into a list of numbers and count the numbers
count = [len(line.strip('{}').split(', ')) for line in data]
print(count)

[2003]
[2003]
[2003]


In [210]:
# Load and preprocess the blank images
#0: dot, 1: circle, 2: ex, 3: plus, 4: blank
blank_images = []
blank_image_files = [file for file in os.listdir(samples_dir[1]) if 'blank' in file]

for image_file in blank_image_files:
    img = Image.open(os.path.join(samples_dir[1], image_file)).convert('L')
    img = np.array(img)
    img = img[4:11, 4:11] / 255.0  # Trim to 7x7 centered and normalize pixel values
    blank_images.append(img)

blank_images = np.array(blank_images)

# Predict the class of the blank images
blank_predictions = model.predict(blank_images)
print(np.round(blank_predictions[:,:-1]*255).astype(int))
# Print the predictions
for i, prediction in enumerate(blank_predictions):
    print(f'Blank image {i+1}: {np.argmax(prediction)}')

1/1 [==============================] - 0s 20ms/step
[[  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]
 [  0 156  64  68]]
Blank image 1: 4
Blank image 2: 4
Blank image 3: 4
Blank image 4: 4
Blank image 5: 4
Blank image 6: 4
Blank image 7: 4
Blank image 8: 4
Blank image 9: 4
Blank image 10: 4
Blank image 11: 4
Blank image 12: 4
Blank image 13: 4
Blank image 14: 4
Blank image 15: 4
Blank image 16: 4
Blank image 17: 4
Blank image 18: 4
Blank image 19: 4
Blank image 20: 4
Blank image 21: 4
Blank image 22: 4
Blank image 23: 4
Blank image 24: 4
Blank image